In [1]:
%load_ext line_profiler

import csv
from collections import defaultdict
from tqdm import tqdm
from random import sample,shuffle
import random
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import model_selection
from sklearn import feature_selection
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn import pipeline
from sklearn import feature_extraction
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.model_selection import learning_curve
from sklearn.neural_network import MLPClassifier as MLP
import numpy as np
import matplotlib.pyplot as plt
from numpy import dot
from numpy.linalg import norm
from scipy.sparse import csr_matrix,lil_matrix,identity

import pandas
import multiprocessing as mp

In [2]:
### In[2]:
random.seed(4)

# Read in list of edges from csv
def load_csv(fname="train.txt"):
    with open(fname) as file:
        reader = csv.reader(file, delimiter="\t")
        rows = []
        for row in tqdm(reader, total=20000, desc="reading rows"):
            row = [int(el) for el in row]
            source = row[0]
            sinks = sorted(row[1:])
            rows.append((source, sinks))
            
        rows.sort(key=lambda row: row[0])
        
        edges = []
        for row in tqdm(rows, total=20000, desc="converting to edges"):
            source, sinks = row
            edges += [(source, sink) for sink in sorted(sinks)]

        return edges

edges = load_csv()

converting to edges: 100%|██████████| 20000/20000 [00:06<00:00, 3093.86it/s]


In [3]:
print("sorting edges")

# Randomly generate new edges, based on a set of existing edges
def random_edges(edges, n):
    return [(random.choice(edges)[0], random.choice(edges)[1]) for i in range(n)]

n_train = 60000
n_val = 2000

print("splitting")
# Keep 1000 edges for training and validation
# the rest is "prior knowledge"
kn, ukn = next(model_selection.ShuffleSplit(test_size=(n_train + n_val)//2).split(edges))
print("sorting")
kn.sort()
ukn.sort()
print("getting")
edges_known = [edges[i] for i in kn]
edges_unknown = [edges[i] for i in ukn]
# edges_known, edges_unknown = model_selection.train_test_split(edges, test_size=(n_train + n_val)//2)

# Keep 200 of the 1000 for validation
edges_train, edges_val = model_selection.train_test_split(edges_unknown, test_size=n_val//2)

print("adding negative cases")
edges_train = edges_train + random_edges(edges_known, len(edges_train))
edges_val   = edges_val + random_edges(edges_val, len(edges_val))
shuffle(edges_train)
shuffle(edges_val)

print("known: {}, training: {}, validation: {}".format(len(edges_known), len(edges_train), len(edges_val)))

sorting edges
splitting
sorting
getting
adding negative cases
known: 23973361, training: 60000, validation: 2000


In [4]:
MAX_ID=4867135
def process(edges, desc="", max_id=MAX_ID):
    adjacency = lil_matrix((max_id+1, max_id+1))
    for (source, sink) in tqdm(edges, desc="processing " + desc + " edges"):
        adjacency[source, sink] = 1
    return adjacency.tocsr(), adjacency

print("starting known edges")
adjacency_known, adjacency_known_lil = process(edges_known, "known")
print("transposing known")
adjacency_t_known = adjacency_known.transpose(copy=True)
adjacency_t_known_lil = adjacency_known_lil.transpose(copy=True)
print("processing unknown edges")
adjacency_unknown, _ = process(edges_unknown, "unknown")
print("adding known to unknown")
adjacency = adjacency_known + adjacency_unknown


starting known edges


processing known edges: 100%|██████████| 23973361/23973361 [01:15<00:00, 318937.61it/s]


transposing known
processing unknown edges


processing unknown edges: 100%|██████████| 31000/31000 [00:00<00:00, 285129.08it/s]


adding known to unknown


In [5]:
def friends_following(source, sink, friends_are_followers=False):
    # Get who source follows, and who follows sink, do a dot product to count
    # the number of elements where both are non-zero
    if friends_are_followers:
        return len(set(adjacency_t_known_lil[source].tocsr().indices).intersection(adjacency_t_known_lil[sink].tocsr().indices))
    else:
        return len(set(adjacency_known[source].tocsr().indices).intersection(adjacency_t_known_lil[sink].tocsr().indices))

def jaccard(a, b):
    a = set(a.indices)
    b = set(b.indices)
    union = len(a.union(b))
    if union == 0:
        return np.nan
    return len(a.intersection(b)) / union

def cosine(a, b):
    return a.dot(b.transpose())[0,0] / np.sqrt(a.count_nonzero() + b.count_nonzero())
    return cosine_similarity(a, b)[0, 0]

def interactions(features, func=np.multiply, infix="*"):
    return { k1 + infix + k2: func(v1, v2) for k1,v1 in features.items() for k2,v2 in features.items() if k1 != k2 }

def derived(features, func=np.reciprocal, prefix = "1/", postfix = ""):
    return { prefix + k + postfix: func(v) for k,v in features.items()}

def safe_div(a, b):
    return a / (b + 1)

def do_sample(sparse, n):
    rows, cols = sparse.nonzero()
    els = list(cols)
    if n >= len(els):
        return els
    return sample(els, n)

def intersection_count(a, b):
    return len(set(a.indices).intersection(set(b.indices)))

def safe_arr_op(f):
    def safe_f(arr):
        if len(arr):
            return f(arr)
        return np.nan
    return safe_f

# Encode the training "X" data and "y" value of a data point (edge)
# a is source, b is sink
def make_row(a, b):
    a_followees = adjacency_known[a]
    b_followees = adjacency_known[b]
    a_followers = adjacency_t_known_lil[a].tocsr()
    b_followers = adjacency_t_known_lil[b].tocsr()
    
    friends = {
        "a_followees": a_followees,
        "b_followees": b_followees,
        "a_followers": a_followers,
        "b_followers": b_followers,
    }
    
    counts = { "n_" + key: val.count_nonzero() for (key, val) in friends.items() }
    count_ratios = interactions(counts, func=safe_div, infix="/")
    
    intersection_counts = interactions(friends, func=intersection_count, infix="`intersection_count`")

    jaccards = interactions({
        "a_followees": a_followees,
        "b_followees": b_followees,
        "a_followers": a_followers,
        "b_followers": b_followers
    }, func=jaccard, infix="`jaccard`")

    cosines = interactions({
        "a_followees": a_followees,
        "b_followees": b_followees,
        "a_followers": a_followers,
        "b_followers": b_followers
    }, func=cosine, infix="`cosine`")

    sample_followees = do_sample(a_followees, 5)
    followees_followees = [adjacency_known[followee] for followee in sample_followees]
    followees_followers = [adjacency_t_known_lil[followee].tocsr() for followee in sample_followees]
    sample_followees_followee_jaccards = [jaccard(b_followees, followee) for followee in followees_followees]
    sample_followees_follower_jaccards = [jaccard(b_followers, follower) for follower in followees_followers]
    sample_followees_followee_cosines  = [cosine(b_followees,  followee) for followee in followees_followees]
    sample_followees_follower_cosines  = [cosine(b_followers,  follower) for follower in followees_followers]
    sample_followees_followee_counts   = [followee.count_nonzero() for followee in followees_followees]
    sample_followees_follower_counts   = [follower.count_nonzero() for follower in followees_followers]

    sample_followers = do_sample(b_followers, 5)
    followers_followees = [adjacency_known[follower] for follower in sample_followers]
    followers_followers = [adjacency_t_known_lil[follower].tocsr() for follower in sample_followers]
    sample_followers_followee_jaccards = [jaccard(a_followees, followee) for followee in followers_followees]
    sample_followers_follower_jaccards = [jaccard(a_followers, follower) for follower in followers_followers]
    sample_followers_followee_cosines  = [cosine(a_followees,  followee) for followee in followers_followees]
    sample_followers_follower_cosines  = [cosine(a_followers,  follower) for follower in followers_followers]
    sample_followers_followee_counts   = [followee.count_nonzero() for followee in followers_followees]
    sample_followers_follower_counts   = [follower.count_nonzero() for follower in followers_followers]

    sampled = {
        func_name + "(" + arr_name + ")": func(arr)
        for func_name, func in [("mean", np.mean), ("std", np.std), ("var", np.var), ("median", np.median), ("min", safe_arr_op(np.nanmin)), ("max", safe_arr_op(np.nanmax))]
        for arr_name, arr in [
            ("sample_followees_followee_jaccards", sample_followees_followee_jaccards),
            ("sample_followees_follower_jaccards", sample_followees_follower_jaccards),
            ("sample_followees_followee_cosines",  sample_followees_followee_cosines),
            ("sample_followees_follower_cosines",  sample_followees_follower_cosines),
            ("sample_followees_followee_counts",   sample_followees_followee_counts),
            ("sample_followees_follower_counts",   sample_followees_follower_counts),
            ("sample_followers_followee_jaccards", sample_followers_followee_jaccards),
            ("sample_followers_follower_jaccards", sample_followers_follower_jaccards),
            ("sample_followers_followee_cosines",  sample_followers_followee_cosines),
            ("sample_followers_follower_cosines",  sample_followers_follower_cosines),
            ("sample_followers_followee_counts",   sample_followers_followee_counts),
            ("sample_followers_follower_counts",   sample_followers_follower_counts),
        ]
    }
    
    followee_friends_that_are_followers = friends_following(a, b)
    followee_friends_that_are_followees = friends_following(b, a)
    follower_friends_that_are_followers = friends_following(a, b, True)
    follower_friends_that_are_followees = friends_following(b, a, True)
    follows_back = adjacency_known[b, a]

    return {
        **counts,
        **intersection_counts,
        **count_ratios,
        **jaccards,
        **cosines,
        **sampled,
        "follows_back": follows_back,
        "followee_friends_that_are_followers": followee_friends_that_are_followers,
        "followee_friends_that_are_followees": followee_friends_that_are_followees,
        "follower_friends_that_are_followers": follower_friends_that_are_followers,
        "follower_friends_that_are_followees": follower_friends_that_are_followees,
    }


# Get the correct y value for a data point
def get_y(source, sink, validation = False):
    if validation:
        return 1
    return 1 if adjacency[source, sink] else 0

# Given a list of edges, create X and y matrices of features
def get_features_(edges, pos, desc="", validation=False):
    X = []
    y = []

    for (source, sink) in tqdm(edges, desc=desc + " features " + str(pos), position=pos) if pos==0 else edges:
        X_row = make_row(source, sink)
        X.append(X_row)
        y.append(get_y(source, sink, validation))
    
    return X, y

def do_row(edge):
    return make_row(edge[0], edge[1])
def do_y(edge):
    return get_y(edge[0], edge[1])

def chunks(l, n):
    ret = [l[i*len(l)//n:(i+1)*len(l)//n] for i in range(n)]
    assert(np.sum([len(chunk) for chunk in ret]) == len(l))
    return ret


def do_chunk(chunk):
    return get_features_(chunk[1], pos=chunk[0])

def do_chunk_val(chunk):
    return get_features_(chunk[1], pos=chunk[0])

# Given a list of edges, create X and y matrices of features
def get_features(edges, desc="", validation=False):
  
    pool = mp.Pool()
    X = []
    y = []
    edges_chunked = chunks(edges, 4)
    for res in pool.map(do_chunk_val, enumerate(edges_chunked)) if validation else pool.map(do_chunk, enumerate(edges_chunked)):
        X += res[0]
        y += res[1]
    
    return X, y

# Get some feature for a row, so we can see an example
row = get_features_([edges_train[8]], pos=0)[0][0]
n_features=len(row)

print("number of features before polynomial interactions: {}".format(n_features))
row

 features 0:   0%|          | 0/1 [00:00<?, ?it/s]/usr/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/lib/python3.6/site-packages/numpy/lib/function_base.py:4033: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
 features 0: 100%|██████████| 1/1 [00:00<00:00,  1.37it/s]

number of features before polynomial interactions: 129


{'n_a_followees': 51725,
 'n_b_followees': 0,
 'n_a_followers': 976,
 'n_b_followers': 146,
 'a_followees`intersection_count`b_followees': 0,
 'a_followees`intersection_count`a_followers': 931,
 'a_followees`intersection_count`b_followers': 117,
 'b_followees`intersection_count`a_followees': 0,
 'b_followees`intersection_count`a_followers': 0,
 'b_followees`intersection_count`b_followers': 0,
 'a_followers`intersection_count`a_followees': 931,
 'a_followers`intersection_count`b_followees': 0,
 'a_followers`intersection_count`b_followers': 115,
 'b_followers`intersection_count`a_followees': 117,
 'b_followers`intersection_count`b_followees': 0,
 'b_followers`intersection_count`a_followers': 115,
 'n_a_followees/n_b_followees': 51725.0,
 'n_a_followees/n_a_followers': 52.94268167860798,
 'n_a_followees/n_b_followers': 351.8707482993197,
 'n_b_followees/n_a_followees': 0.0,
 'n_b_followees/n_a_followers': 0.0,
 'n_b_followees/n_b_followers': 0.0,
 'n_a_followers/n_a_followees': 0.01886865

In [6]:
def test():
    get_features_(edges_train[:60], pos=0)
%lprun -f test -f make_row -f jaccard -f cosine test()

get_features(edges_train[:60])[0][0]

None

 features 0:   2%|▏         | 1/60 [00:00<00:33,  1.76it/s]/usr/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3194: Runtim

In [ ]:
random.seed(5)
# Make our final training data, by sampling from existing edges, and adding in other random edges
# We sample a small subset from our training data for computational reasons
X_train, y_train = get_features(edges_train, desc="train")
X_val, y_val = get_features(edges_val, desc="validation")

# Frequency count to check how unbalanced our classes are
print(pandas.Series(y_train).value_counts())
print(pandas.Series(y_val).value_counts())

 features 0:   0%|          | 0/15000 [00:00<?, ?it/s]/usr/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
 features 0:   0%|          | 1/15000 [00:01<4:19:30,  1.04s/it]/usr/lib/python3.6/site-packages/numpy/lib/function_base.py:4033: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/lib/python3.6/site-packages/numpy/lib/function_base.py:4033: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: All-NaN axis encountered
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/lib/python3.6/sit

In [ ]:
random.seed(1)
def inv(X):
    return np.hstack([X, 1 / (X + 1), X==0, ])

def add_is_zero(X):
    return np.hstack([X, X==0])

model = Pipeline([
    ('vec', feature_extraction.DictVectorizer(sparse=False)),
    ('impute', preprocessing.Imputer()),
#     ('booleans', preprocessing.FunctionTransformer(add_is_zero)),
    ('norm', preprocessing.StandardScaler()),
#     ('model', MLP(hidden_layer_sizes=n_features)),
#     ('select', feature_selection.RFE(LR(), n_features_to_select=100, verbose=3)),
    ('select', feature_selection.SelectKBest(k=100)),
    ('model', MLP()),
])

train_sizes, train_scores, val_scores = learning_curve(model, X_train, y_train, train_sizes=np.arange(0.05, 1.01, 0.05), cv=5)

train_scores = [np.mean(scores) for scores in train_scores]
val_scores = [np.mean(scores) for scores in val_scores]

print(train_scores)
print(val_scores)
plt.plot(train_sizes, val_scores, 'b', label='test')
plt.plot(train_sizes, train_scores, 'r', label='train')
plt.show()

In [ ]:
random.seed(2)
model.fit(X_train, y_train)

In [ ]:
random.seed(3)
# Check how we score against the validation set
print(model.score(X_val, y_val))
print(metrics.roc_auc_score(model.predict(X_val), y_val))
print(metrics.classification_report(model.predict(X_val), y_val))

# Check how many accurate predictions we have for each class
print(confusion_matrix(model.predict(X_val), y_val))

In [ ]:
# Read in the test data, make predictions with a model, write to a new csv
def make_submission(model, file_in="test-public.txt", file_out="predictions.csv"):
    edges = []
    with open(file_in) as file:
        reader = csv.reader(file, delimiter="\t")
        header = True
        for row in reader:
            if header:
                header = False
                continue
            row = [int(el) for el in row]
            id, source, sink = row
            edges.append(tuple([source, sink]))
            
    X_test, _ = get_features(edges, desc="pred", validation=True)
    y_pred = model.predict_proba(X_test)
        
    with open(file_out, 'w') as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(["Id", "Prediction"])
        for i in range(len(y_pred)):
            writer.writerow([i+1, y_pred[i][1]])

make_submission(model)